# Evaluating AtsPy with Population Clustering

## Environment Setup

When running this notebook on Google Colab, we may have to NB Load The Package, then "Runtime" -> "Restart Runtime" For everything To load correctly.

In [ ]:
!pip install atspy
!pip install pigar
!pip install holidays==0.9.5

In [ ]:
import numpy as np
import pandas as pd
from atspy import AutomatedModel
from scipy import stats
import scipy.sparse as sparse
import scipy.sparse.csgraph as csgraph

## Generating Clusters

In [ ]:
data = pd.read_csv('../data/graph.csv')
data.head()

In [ ]:
data.describe()

In [ ]:
# Given a single data record, returns the total flow as a time series. 
def extract_timeseries(point):
    ts = pd.Series(point[2:])
    ts.index = pd.to_datetime(point.index[2:])
    return ts

In [ ]:
states = np.unique(data['source_state'])

# Adjacency Matrix is a 50x50 array. 
# adjacency[s, t] = # people going from source state s to target state t
# s and t are state "numbers"; ie. the index of the state in "states"
adj = np.zeros((len(states), len(states)))

for s, source_state in enumerate(states): 
    src_data = data.loc[data['source_state'] == source_state]
    for t, target_state in enumerate(states): 
        entry = src_data.loc[src_data['target_state'] == target_state].iloc[0]     
        adj[s, t] = np.sum(extract_timeseries(entry))

assert adj[0, 0] == extract_timeseries(data.iloc[0]).sum()

In [ ]:
ε = 5e6

# connected components; 1 if connection, 0 if not. 
conn = np.zeros((len(states), len(states)))
conn[adj > ε] = 1
conn[adj <= ε] = 0 

print(conn)

In [ ]:
n, clusters = csgraph.connected_components(
    csgraph=sparse.csr_matrix(conn), 
    connection='weak'
    directed=False, 
    return_labels=True)

In [ ]:
print(n)
print(clusters)

In [ ]:
for label in np.unique(clusters): 
    print(f'Cluster {label}:')
    for ind, val in enumerate(clusters): 
        if val == label: 
            print(f'\t{states[ind]}')
    print()

## Creating Pseudo-States

Here, we've got to augment the data of each individual state with the data from their cluster.

We previously tried to do this using arithmetic averages. That didn't work. Now we need to be smarter.


In [ ]:
#

## Using Clustering with AtsPy

AtsPy allows for several models. The pertinent model codes are:

1. ```ARIMA``` - Automated ARIMA Modelling
1. ```HWAAS``` - Exponential Smoothing With Additive Trend and Additive Seasonality
1. ```HWAMS``` - Exponential Smoothing with Additive Trend and Multiplicative Seasonality

Note that the `HWAAS` and `HWAMS` are variants of the Holt-Winters algorithm. 

In [ ]:
df = pd.read_csv("../data/train.csv")
cal = pd.Series.to_frame(df[df['Province_State'] == 'California']['Confirmed'])
cal['Date'] = pd.to_datetime(df.Date)
cal = cal.set_index("Date")
train = cal[0:112]
train

In [ ]:
# HWAMS doesn't seem to do as well :(
model_list=["HWAAS"]
results = {}
test = {}
for state, group in df.groupby('Province_State'):
  print("Training", state)
  split = pd.Series.to_frame(group['Confirmed'])
  split['Date'] = pd.to_datetime(group.Date)
  split = split.set_index("Date")
  train = split[:112]
  test[state] = split[112:]['cl'].values * cl[state]
  am = AutomatedModel(df = train, model_list=model_list, forecast_len=30 )
  forecast_out = am.forecast_outsample()
  results[state] = forecast_out['HWAAS'].values

  # for state, group in clf.groupby('Province_State'):
  #   print("Training", state)
  #   if state not in np.unique(df['Province_State']):
  #     split = pd.Series.to_frame(group['Confirmed'])
  #     split['Date'] = pd.to_datetime(group.Date)
  #     split = split.set_index("Date")
  #     train = split[:112]
  #     test[state] = split[112:]['cl'].values * cl[state]
  #     am = AutomatedModel(df = train, model_list=model_list, forecast_len=30 )
  #     forecast_out = am.forecast_outsample()
  #     results[state] = forecast_out['ARIMA'].values

In [ ]:
results

In [ ]:
def MAPE(predicted, actual):
    assert len(predicted) == len(actual)
    res = 0
    for i in range(len(predicted)):
        diff = np.abs(predicted[i] - actual[i]) / np.abs(actual[i])
        res += diff
    return (res/len(predicted)) * 100

In [ ]:
mapes = []
for state in results:
  pred = results[state]
  t = test[state]
  mape = MAPE(pred, t)
  print(state, mape)
  mapes.append(mape)

In [ ]:
total_mape = 0
for mape in mapes:
  total_mape += mape*30
total_mape = total_mape/1500
total_mape

In [ ]:
def train_full(df, model_name, param):
  results = {}
  for state, group in df.groupby('Province_State'):
    print("Training", state)
    split = pd.Series.to_frame(group[param])
    split['Date'] = pd.to_datetime(group.Date)
    split = split.set_index("Date")
    am = AutomatedModel(df = split, model_list=[model_name], forecast_len=26 )
    forecast_out = am.forecast_outsample()
    results[state] = forecast_out[model_name].values
  return results

In [ ]:
conf = train_full(df, 'HWAAS', "Confirmed")

In [ ]:
conf

In [ ]:
death = train_full(df, 'HWAAS', 'Deaths')

In [ ]:
death

In [ ]:
test = pd.read_csv('test.csv')


for index, state in enumerate(np.unique(df['Province_State'])):
    predicted_cases = conf[state]
    for j in range(len(predicted_cases)):
        cur_index = index + j * 50
        test['Confirmed'].iloc[cur_index] = predicted_cases[j]


for index, state in enumerate(np.unique(df['Province_State'])):
    predicted_cases = death[state]
    for j in range(len(predicted_cases)):
        cur_index = index + j * 50
        test['Deaths'].iloc[cur_index] = predicted_cases[j]


submission = test
submission = submission.drop(['Province_State', 'Date'], axis = 1)
submission.head()

submission.to_csv('holt_sub.csv', index=False)

## Redividing States

Here, we've got to split the clusters back into their component states. 

We previously tried to do this using arithmetic averages. 
That didn't work. 
Now we need to be smarter. 

In [ ]:
#